# 5. Searching KItems with the SDK

In this tutorial we see how to search existing Kitems

### 5.1. Setting up
Before you run this tutorial: make sure to have access to a DSMS-instance of your interest, alongwith with installation of this package and have establised access to the DSMS through DSMS-SDK (refer to [Connecting to DSMS](../dsms_sdk.md#connecting-to-dsms))


Now let us import the needed classes and functions for this tutorial.

In [1]:
from dsms import DSMS, KItem

Now source the environmental variables from an `.env` file and start the DSMS-session.

In [2]:
dsms = DSMS(env=".env")

### 5.2. Searching for KItems

In this section, we would like to search for specfic KItems we created in the DSMS.

For this purpose, we will firstly create some KItems and apply the `search`-method on the `DSMS`-object later on in order to find them again in the DSMS.

We also want to demonstrate here, that we can link KItems to each other in order to find e.g. a related item of type `DatasetCatalog`. For this strategy, we are using the `linked_kitems`- attribute and the `id` of the item which we would like to link.

The procedure looks like this:

In [3]:
item1 = KItem(
    name="Machine-1",
    ktype_id=dsms.ktypes.Expert,
    annotations=["https://w3id.org/steel/ProcessOntology/TestingMachine"],
    custom_properties={"Producer": "TestingLab GmBH",
                       "Room Number": "A404",
                       "Description": "Bending Test Machine"
                       }
)

item2 = KItem(
    name="Machine-2",
    ktype_id=dsms.ktypes.Testingmachine,
    annotations=["https://w3id.org/steel/ProcessOntology/TestingMachine"],
    custom_properties={"Producer": "StressStrain GmBH",
                       "Room Number": "B500",
                       "Description": "Compression Test Machine"
                       }
)

item3 = KItem(
    name="Specimen-1", 
    ktype_id=dsms.ktypes.Specimen,
    linked_kitems=[item1],
    annotations=["https://w3id.org/steel/ProcessOntology/TestPiece"],
    custom_properties = {
        "Width": 0.5,
        "Length": [0.1, 0.2],
    }

)
item4 = KItem(
    name="Specimen-2",
    ktype_id=dsms.ktypes.Specimen,
    linked_kitems=[item2],
    annotations=["https://w3id.org/steel/ProcessOntology/TestPiece"],
    custom_properties = {
        "Width": 0.8,
        "Length": [0.8, 0.9],
    }
)

item5 = KItem(
    name="Research Institute ABC",
    ktype_id=dsms.ktypes.Organization,
    linked_kitems=[item1,item2],
    annotations=["www.researchBACiri.org/foo"],
)

dsms.add([item1, item2, item3, item4, item5])
dsms.commit()

/app/dsms/knowledge/kitem.py:512: UserWarning: A flat dictionary was provided for custom properties.
                    Will be transformed into `KItemCustomPropertiesModel`.
  warnings.warn(
/app/dsms/knowledge/kitem.py:412: UserWarning: Found a <class 'dsms.knowledge.kitem.KItem'> to be linked instead of an <class 'dsms.knowledge.properties.linked_kitems.KItemRelationshipModel'>. Will link it with the default relationship 'dcterms:haspart'.
  warnings.warn(


ValidationError: 2 validation errors for KItem
linked_kitems.kitem.LinkedKItem
  Input should be a valid dictionary or instance of LinkedKItem [type=model_type, input_value=kitem:
  name: Machine-1
...        required: false
, input_type=KItem]
    For further information visit https://errors.pydantic.dev/2.11/v/model_type
linked_kitems.kitem.dict[any,any]
  Input should be a valid dictionary [type=dict_type, input_value=kitem:
  name: Machine-1
...        required: false
, input_type=KItem]
    For further information visit https://errors.pydantic.dev/2.11/v/dict_type


<b> <p style="color:red;">Note : Here in this tutorial, we use dsms.search with `limit=1` to maintain readability but the user can adjust the variable `limit` as per requirement. </p>    </b> 


Now, we are apply to search for e.g. kitems of type `TestingMachine`:

In [ ]:
result = dsms.search(ktypes=[dsms.ktypes.Specimen], limit=2)
print(result)
print("Name of the first kitem:")
print(result.hits[0].kitem.name)

hits:
- kitem:
    id: a433c4f1-0e5a-4dcd-84e8-67ead16204bc
    name: Specimen-1
    ktype_id: specimen
    slug: specimen-1-a433c4f1
    annotations:
    - iri: https://w3id.org/steel/ProcessOntology/TestPiece
      label: TestPiece
      namespace: https://w3id.org/steel/ProcessOntology
    attachments:
    - name: subgraph.ttl
    linked_kitems:
    - is_incoming: false
      label: Has Part
      kitem:
        id: 5124c781-9bff-4104-8888-886e81d735de
        name: Machine-1
        slug: machine-1-5124c781
        ktype_id: expert
        summary: null
        avatar_exists: false
        annotations:
        - iri: https://w3id.org/steel/ProcessOntology/TestingMachine
          label: TestingMachine
          namespace: https://w3id.org/steel/ProcessOntology
        linked_kitems: []
        external_links: []
        contacts: []
        authors:
        - user_id: 7f0e5a37-353b-4bbc-b1f1-b6ad575f562d
        affiliations: []
        attachments: []
        user_groups: []
     

... and for all of type `Organization` and `Testingmachine`:

In [ ]:
for result in dsms.search(ktypes=[dsms.ktypes.Organization, dsms.ktypes.Expert], limit=2):
    print(result.kitem)
    print("fuzziness: ", result.fuzzy)
    print("\n")
    

kitem:
  id: 5124c781-9bff-4104-8888-886e81d735de
  name: Machine-1
  ktype_id: expert
  slug: machine-1-5124c781
  annotations:
  - iri: https://w3id.org/steel/ProcessOntology/TestingMachine
    label: TestingMachine
    namespace: https://w3id.org/steel/ProcessOntology
  attachments: []
  linked_kitems:
  - is_incoming: true
    label: Has Part
    kitem:
      id: 9dd75024-91a9-4b18-a602-54517898cebb
      name: Research Institute ABC
      slug: researchinstituteabc-9dd75024
      ktype_id: organization
      summary: null
      avatar_exists: false
      annotations:
      - iri: www.researchBACiri.org/foo
        label: foo
        namespace: www.researchBACiri.org
      linked_kitems: []
      external_links: []
      contacts: []
      authors:
      - user_id: 7f0e5a37-353b-4bbc-b1f1-b6ad575f562d
      affiliations: []
      attachments:
      - name: subgraph.ttl
      user_groups: []
      custom_properties: null
      apps: []
      created_at: '2025-07-22 13:34:19.596915'


... or for all of type `Specimen` with `Specimen-1` in the name (Please note that the `allow_fuzzy` is set the `False` in order to get only exact matches):

In [ ]:
for result in dsms.search(query="Specimen-1", ktypes=[dsms.ktypes.Specimen], allow_fuzzy=False, limit=1):
    print(result)

kitem:
  id: a433c4f1-0e5a-4dcd-84e8-67ead16204bc
  name: Specimen-1
  ktype_id: specimen
  slug: specimen-1-a433c4f1
  annotations:
  - iri: https://w3id.org/steel/ProcessOntology/TestPiece
    label: TestPiece
    namespace: https://w3id.org/steel/ProcessOntology
  attachments:
  - name: subgraph.ttl
  linked_kitems:
  - is_incoming: false
    label: Has Part
    kitem:
      id: 5124c781-9bff-4104-8888-886e81d735de
      name: Machine-1
      slug: machine-1-5124c781
      ktype_id: expert
      summary: null
      avatar_exists: false
      annotations:
      - iri: https://w3id.org/steel/ProcessOntology/TestingMachine
        label: TestingMachine
        namespace: https://w3id.org/steel/ProcessOntology
      linked_kitems: []
      external_links: []
      contacts: []
      authors:
      - user_id: 7f0e5a37-353b-4bbc-b1f1-b6ad575f562d
      affiliations: []
      attachments: []
      user_groups: []
      custom_properties:
        sections:
        - id: id1753191254948dr7y4

... and for all of type `Organization` with the annotation `www.researchBACiri.org/foo`:

In [ ]:
for result in dsms.search(
          ktypes=[dsms.ktypes.Organization], annotations=["www.researchBACiri.org/foo"], allow_fuzzy=False, limit=1
    ):
    print(result)


kitem:
  id: 9dd75024-91a9-4b18-a602-54517898cebb
  name: Research Institute ABC
  ktype_id: organization
  slug: researchinstituteabc-9dd75024
  annotations:
  - iri: www.researchBACiri.org/foo
    label: foo
    namespace: www.researchBACiri.org
  attachments:
  - name: subgraph.ttl
  linked_kitems:
  - is_incoming: false
    label: Has Part
    kitem:
      id: 2d4e7254-d11f-49e7-973b-bcfcdfc89d79
      name: Machine-2
      slug: machine-2-2d4e7254
      ktype_id: testingmachine
      summary: null
      avatar_exists: false
      annotations:
      - iri: https://w3id.org/steel/ProcessOntology/TestingMachine
        label: TestingMachine
        namespace: https://w3id.org/steel/ProcessOntology
      linked_kitems: []
      external_links: []
      contacts: []
      authors:
      - user_id: 7f0e5a37-353b-4bbc-b1f1-b6ad575f562d
      affiliations: []
      attachments: []
      user_groups: []
      custom_properties:
        sections:
        - id: id1753191254980fgfnya
        

## 5.3. Fetching linked KItems from a KItem

In the beginning under **5.1** we created some kitems and linked each other. Now we want to fetch the linked kitems and display them to the user. For this we use the `linked_kitems` attribute.

In [ ]:
item5.linked_kitems

[KItemRelationshipModel(is_incoming=False, label='Has Part', kitem=linked_kitem:
   id: 2d4e7254-d11f-49e7-973b-bcfcdfc89d79
   name: Machine-2
   slug: machine-2-2d4e7254
   ktype_id: testingmachine
   summary: null
   avatar_exists: false
   annotations:
   - iri: https://w3id.org/steel/ProcessOntology/TestingMachine
     label: TestingMachine
     namespace: https://w3id.org/steel/ProcessOntology
   linked_kitems: []
   external_links: []
   contacts: []
   authors:
   - user_id: 7f0e5a37-353b-4bbc-b1f1-b6ad575f562d
   affiliations: []
   attachments: []
   user_groups: []
   custom_properties:
     sections:
     - id: id1753191254980fgfnya
       name: Misc
       entries:
       - id: id1753191254980sqd116
         type: Text
         label: Producer
         value: StressStrain GmBH
         required: false
       - id: id1753191254980tqrmuw
         type: Text
         label: Room Number
         value: B500
         required: false
       - id: id1753191254980oz5zjx
         t

However, this linked KItem is not a "real" KItem. Due to performance reasons, we only display a slim representation of the linked KItem. Imagine if the linked KItem also has a linked KItem... this might lead to an infinite recursion. We can fetch the "real" KItem by using the `fetch` method of the linked KItem:


In [ ]:
item5.linked_kitems[0].fetch()

kitem:
  id: 2d4e7254-d11f-49e7-973b-bcfcdfc89d79
  name: Machine-2
  ktype_id: testingmachine
  slug: machine-2-2d4e7254
  annotations:
  - iri: https://w3id.org/steel/ProcessOntology/TestingMachine
    label: TestingMachine
    namespace: https://w3id.org/steel/ProcessOntology
  attachments: []
  linked_kitems: []
  affiliations: []
  authors:
  - user_id: 7f0e5a37-353b-4bbc-b1f1-b6ad575f562d
  avatar_exists: false
  contacts: []
  created_at: 2025-07-22 13:34:16.187201
  updated_at: 2025-07-22 13:34:16.187201
  external_links: []
  apps: []
  user_groups: []
  custom_properties:
    content:
      sections:
      - id: id1753191254980fgfnya
        name: Misc
        entries:
        - id: id1753191254980sqd116
          type: Text
          label: Producer
          value: StressStrain GmBH
          measurementUnit: null
          relationMapping: null
          required: false
        - id: id1753191254980tqrmuw
          type: Text
          label: Room Number
          value: B

We can also get the linked KItems grouped by annotation:

In [ ]:
item5.linked_kitems.by_annotation

{'https://w3id.org/steel/ProcessOntology/TestingMachine': [KItemRelationshipModel(is_incoming=False, label='Has Part', kitem=linked_kitem:
    id: 2d4e7254-d11f-49e7-973b-bcfcdfc89d79
    name: Machine-2
    slug: machine-2-2d4e7254
    ktype_id: testingmachine
    summary: null
    avatar_exists: false
    annotations:
    - iri: https://w3id.org/steel/ProcessOntology/TestingMachine
      label: TestingMachine
      namespace: https://w3id.org/steel/ProcessOntology
    linked_kitems: []
    external_links: []
    contacts: []
    authors:
    - user_id: 7f0e5a37-353b-4bbc-b1f1-b6ad575f562d
    affiliations: []
    attachments: []
    user_groups: []
    custom_properties:
      sections:
      - id: id1753191254980fgfnya
        name: Misc
        entries:
        - id: id1753191254980sqd116
          type: Text
          label: Producer
          value: StressStrain GmBH
          required: false
        - id: id1753191254980tqrmuw
          type: Text
          label: Room Number
  

Again, we are able to fetch the "real" KItem by using the `fetch` method of the linked KItem:

In [ ]:
item5.linked_kitems.by_annotation["https://w3id.org/steel/ProcessOntology/TestingMachine"][0].fetch()

kitem:
  id: 2d4e7254-d11f-49e7-973b-bcfcdfc89d79
  name: Machine-2
  ktype_id: testingmachine
  slug: machine-2-2d4e7254
  annotations:
  - iri: https://w3id.org/steel/ProcessOntology/TestingMachine
    label: TestingMachine
    namespace: https://w3id.org/steel/ProcessOntology
  attachments: []
  linked_kitems: []
  affiliations: []
  authors:
  - user_id: 7f0e5a37-353b-4bbc-b1f1-b6ad575f562d
  avatar_exists: false
  contacts: []
  created_at: 2025-07-22 13:34:16.187201
  updated_at: 2025-07-22 13:34:16.187201
  external_links: []
  apps: []
  user_groups: []
  custom_properties:
    content:
      sections:
      - id: id1753191254980fgfnya
        name: Misc
        entries:
        - id: id1753191254980sqd116
          type: Text
          label: Producer
          value: StressStrain GmBH
          measurementUnit: null
          relationMapping: null
          required: false
        - id: id1753191254980tqrmuw
          type: Text
          label: Room Number
          value: B

Additionally, we can retrieve the linked KItems grouped by ktype:

In [ ]:
item5.linked_kitems.by_ktype

{ktype:
   id: testingmachine
   name: TestingMachine
   created_at: '2025-07-22T12:41:50.505566'
   updated_at: '2025-07-22T12:41:50.505566': [linked_kitem:
    id: 2d4e7254-d11f-49e7-973b-bcfcdfc89d79
    name: Machine-2
    slug: machine-2-2d4e7254
    ktype_id: testingmachine
    summary: null
    avatar_exists: false
    annotations:
    - iri: https://w3id.org/steel/ProcessOntology/TestingMachine
      label: TestingMachine
      namespace: https://w3id.org/steel/ProcessOntology
    linked_kitems: []
    external_links: []
    contacts: []
    authors:
    - user_id: 7f0e5a37-353b-4bbc-b1f1-b6ad575f562d
    affiliations: []
    attachments: []
    user_groups: []
    custom_properties:
      sections:
      - id: id1753191254980fgfnya
        name: Misc
        entries:
        - id: id1753191254980sqd116
          type: Text
          label: Producer
          value: StressStrain GmBH
          required: false
        - id: id1753191254980tqrmuw
          type: Text
          la

... and retrieve them by type:

In [ ]:
item5.linked_kitems.by_ktype[dsms.ktypes.Testingmachine][0].fetch()

kitem:
  id: 2d4e7254-d11f-49e7-973b-bcfcdfc89d79
  name: Machine-2
  ktype_id: testingmachine
  slug: machine-2-2d4e7254
  annotations:
  - iri: https://w3id.org/steel/ProcessOntology/TestingMachine
    label: TestingMachine
    namespace: https://w3id.org/steel/ProcessOntology
  attachments: []
  linked_kitems: []
  affiliations: []
  authors:
  - user_id: 7f0e5a37-353b-4bbc-b1f1-b6ad575f562d
  avatar_exists: false
  contacts: []
  created_at: 2025-07-22 13:34:16.187201
  updated_at: 2025-07-22 13:34:16.187201
  external_links: []
  apps: []
  user_groups: []
  custom_properties:
    content:
      sections:
      - id: id1753191254980fgfnya
        name: Misc
        entries:
        - id: id1753191254980sqd116
          type: Text
          label: Producer
          value: StressStrain GmBH
          measurementUnit: null
          relationMapping: null
          required: false
        - id: id1753191254980tqrmuw
          type: Text
          label: Room Number
          value: B

Clean up the DSMS from the tutortial:

In [ ]:
del dsms[item1]
del dsms[item2]
del dsms[item3]
del dsms[item4]
del dsms[item5]

dsms.commit()